In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import deque
import random
import torch.optim as optim
import numpy as np
import gym
import os
import torch
import numpy as np
import random

In [22]:
class Plane:
    def __init__(self, tag, x_b= 0.000001, y_b= 0.000001, z_b= 0.000001, v_x= 0.000001, v_y= 0.000001, v_z= 0.000001, u= 0.000001, v= 0.000001, w= 0.000001, X= 0.000001, Y= 0.000001, Z= 0.000001, T_max=49840, p= 0.000001, q= 0.000001, r= 0.000001, l_a= 0.000001, m_a= 0.000001, n_a= 0.000001, m = 15119):
        self.tag = tag
        self.m = m
        self.x_b = x_b
        self.y_b = y_b
        self.z_b = z_b
        self.v_x = v_x
        self.v_y = v_y
        self.v_z = v_z
        self.u = u
        self.v = v
        self.w = w
        self.X = X
        self.Y = Y
        self.Z = Z
        self.T_max = T_max
        self.p = p
        self.q = q
        self.r = r
        self.p = p
        self.l_a = l_a
        self.m_a = m_a
        self.n_a = n_a
        self.g = 9.8

        # 初始姿态四元数
        self.q_0, self.q_1, self.q_2, self.q_3 = 1,0,0,0
        k = 31184*230414 - (4028.1**2)

        # 飞行器惯量张量
        self.I = np.array([[31184,0,4028.1],[0,205125,0],[4028.1,0,230414]])
        self.I_1 = np.array([[31184/k,0,4028.1/k],[0,1/205125,0],[4028.1/k,0,230414/k]])

        # 飞行器体系下角速度
        self.omega = np.array([[0.000001],[0.000001],[0.000001]])

        # 根据四元数计算欧拉角
        self.phi = 0.000001
        self.theta = 0.000001
        self.psi = 0.000001

        # 计算迎角和侧滑角
        self.alpha = 0.000001
        self.beta = 0.000001

    def update(self,delta_e,delta_a,delta_r,eta,delt_t):
        # 计算迎角和侧滑角
        self.alpha = np.arctan(self.w/self.u)
        self.beta = np.arcsin(self.v/np.sqrt(self.u**2+self.v**2+self.w**2))

        # 更新姿态四元数
        self.q_ = 0.5*np.array([[-self.q_1, -self.q_2, -self.q_3],[self.q_0, -self.q_3, self.q_2],[self.q_3, self.q_0, -self.q_1],[-self.q_2, self.q_1, self.q_0]])@self.omega
        self.q_0 = self.q_[0]
        self.q_1 = self.q_[1]
        self.q_2 = self.q_[2]
        self.q_3 = self.q_[3]

        # 根据四元数计算欧拉角
        self.phi = np.arctan((2*self.q_0*self.q_1+2*self.q_2*self.q_3)/(self.q_0**2-self.q_1**2-self.q_2**2+self.q_3**2))
        self.theta = np.arcsin(2*self.q_0*self.q_2-2*self.q_1*self.q_3)
        self.psi = np.arctan((2*self.q_0*self.q_3+2*self.q_1*self.q_2)/(self.q_0**2+self.q_1**2-self.q_2**2-self.q_3**2))

        # 气动力模型
        if self.alpha>=-5 and self.alpha<20:
            C_D =0.0013*(self.alpha**2)-0.00438*self.alpha+0.1423
        elif self.alpha>=20 and self.alpha<=40:
            C_D = -0.0000348*(self.alpha**2)+0.0473*self.alpha-0.358

        C_Y = -0.0186*self.beta+(delta_a/25)*(-0.00227*self.alpha+0.039)+(delta_r/30)*(-0.00265*self.alpha+0.141)

        if self.alpha>=-5 and self.alpha<10:
            C_L = 0.0751*self.alpha+0.0144*delta_e+0.732
        elif self.alpha>=10 and self.alpha<=40:
            C_L = -0.00148*(self.alpha**2)+0.106*self.alpha+0.0144*delta_e+0.569

        if self.alpha>=-5 and self.alpha<15:
            C_l = (-0.00012*self.alpha-0.00092)*self.beta-0.0315*self.p+0.0126*self.r+(delta_a/25)*(0.00121*self.alpha-0.0628)-(delta_r/30)*(0.000351*self.alpha-0.0124)
        elif self.alpha>=15 and self.alpha<=25:
            C_l = (0.00022*self.alpha-0.006)*self.beta-0.0315*self.p+0.0126*self.r+(delta_a/25)*(0.00121*self.alpha-0.0628)-(delta_r/30)*(0.000351*self.alpha-0.0124)

        C_m = -0.00437*self.alpha-0.0196*delta_e-0.123*self.q-0.1885

        if self.alpha>=-5 and self.alpha<10:
            C_n = 0.00125*self.beta-0.0142*self.r+(delta_a/25)*(0.000213*self.alpha+0.00128)+(delta_r/30)*(0.000804*self.alpha-0.0474)
        elif self.alpha>=10 and self.alpha<25:
            C_n = (-0.00022*self.alpha+0.00342)*self.beta-0.0142*self.r+(delta_a/25)*(0.000213*self.alpha+0.00128)+(delta_r/30)*(0.000804*self.alpha-0.0474)
        elif self.alpha>=25 and self.alpha<=35:
            C_n = -0.00201*self.beta-0.0142*self.r+(delta_a/25)*(0.000213*self.alpha+0.00128)+(delta_r/30)*(0.000804*self.alpha-0.0474)

        rho = 1.293
        V = np.sqrt(self.v_x**2+self.v_y**2+self.v_z)
        Q = 0.5*rho*V**2
        S = 37.16
        b = 11.41
        c = 3.51

        self.D = Q*S*C_D
        self.Y = Q*S*C_Y
        self.L = Q*S*C_L

        self.l_a = Q*S*b*C_l
        self.m_a = Q*S*c*C_m
        self.n_a = Q*S*b*C_m

        # 更新飞行器体系下角速度
        self.omega = self.I_1@(-1*np.array([[0,-self.r,self.q],[self.r,0,-self.p],[-self.q,self.p,0]])@self.I@np.array([[self.p],[self.q],[self.r]])+np.array([[self.l_a],[self.m_a],[self.n_a]]))

        # 方向余弦矩阵
        self.R_g_b = np.array([[np.cos(self.phi),0,-np.sin(self.phi)],[0,1,0],[np.sin(self.phi),0,np.cos(self.phi)]])@np.array([[1,0,0],[0,np.cos(self.theta),np.sin(self.theta)],[0,-np.sin(self.theta),np.cos(self.theta)]])@np.array([[np.cos(self.psi),np.sin(self.psi),0],[-np.sin(self.psi),np.cos(self.psi),0],[0,0,1]])

        self.X = np.cos(self.alpha)*np.cos(self.beta)*-self.D+(-np.cos(self.alpha)*np.sin(self.beta))*self.Y+(-np.sin(self.alpha))*-self.L
        self.Y = np.sin(self.beta)*-self.D+np.cos(self.beta)*self.Y
        self.Z = np.sin(self.alpha)*np.cos(self.beta)*-self.D+(-np.sin(self.alpha)*np.sin(self.beta))*self.Y+(np.cos(self.alpha))*-self.L

        self.u = self.v*self.r-self.w*self.q-self.g*self.R_b_g+(self.X+eta*self.T_max)/self.m
        self.v = -self.u*self.r+self.w*self.p-self.g*self.R_b_g+self.Y/self.m
        self.w = self.u*self.q-self.v*self.p-self.g*self.R_b_g+self.Z/self.m

        self.v_x = self.R_g_b * self.u
        self.v_y = self.R_g_b * self.v
        self.v_z = self.R_g_b * self.w

        self.x_b = delt_t * self.v_x
        self.y_b = delt_t * self.v_y
        self.z_b = delt_t * self.v_z

In [24]:
agent = Plane(1)

In [25]:
agent.update(1,1,1,10,10)

C:\Users\15163\AppData\Local\Temp\ipykernel_13296\3088562803.py:110: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.R_g_b = np.array([[np.cos(self.phi),0,-np.sin(self.phi)],[0,1,0],[np.sin(self.phi),0,np.cos(self.phi)]])@np.array([[1,0,0],[0,np.cos(self.theta),np.sin(self.theta)],[0,-np.sin(self.theta),np.cos(self.theta)]])@np.array([[np.cos(self.psi),np.sin(self.psi),0],[-np.sin(self.psi),np.cos(self.psi),0],[0,0,1]])


AttributeError: 'Plane' object has no attribute 'R_b_g'